In [6]:
import gradio as gr
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [7]:
# Load the trained model
model = joblib.load("/content/drive/MyDrive/best_model.pkl")
geography_encoder = joblib.load("/content/drive/MyDrive/geography_encoder.pkl")
scaler = joblib.load("/content/drive/MyDrive/scaler.pkl")

In [8]:
def predict_churn(
    credit_score,
    geography,
    gender,
    age,
    tenure,
    balance,
    num_of_products,
    has_cr_card,
    is_active_member,
    estimated_salary
):
    # Prepare input data
    input_data = pd.DataFrame([{
        "CreditScore": credit_score,
        "Geography": geography,
        "Gender": 1 if gender == "Male" else 0,
        "Age": age,
        "Tenure": tenure,
        "Balance": balance,
        "NumOfProducts": num_of_products,
        "HasCrCard": has_cr_card,
        "IsActiveMember": is_active_member,
        "EstimatedSalary": estimated_salary
    }])

    # Encode geography using pre-fitted encoder
    geo_encoded = geography_encoder.transform(input_data[["Geography"]])

    # Scale numeric features using pre-fitted scaler
    num_scaled = scaler.transform(input_data[[
        "CreditScore", "Gender", "Age", "Tenure",
        "Balance", "NumOfProducts", "HasCrCard",
        "IsActiveMember", "EstimatedSalary"
    ]])

    # Combine encoded and scaled features
    final_input = np.concatenate((geo_encoded, num_scaled), axis=1)

    # Make prediction
    prediction = model.predict(final_input)[0]
    return "Customer will exit (Churn)" if prediction == 1 else "Customer will stay"


In [9]:
# Gradio interface
interface = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Number(label="Credit Score"),
        gr.Dropdown(["France", "Spain", "Germany"], label="Geography"),
        gr.Radio(["Male", "Female"], label="Gender"),
        gr.Number(label="Age"),
        gr.Number(label="Tenure"),
        gr.Number(label="Balance"),
        gr.Number(label="Number of Products"),
        gr.Checkbox(label="Has Credit Card"),
        gr.Checkbox(label="Is Active Member"),
        gr.Number(label="Estimated Salary"),
    ],
    outputs=gr.Textbox(label="Prediction Result"),
    title="Customer Churn Prediction",
    description="Enter customer information to predict whether they will exit the bank."
)

In [10]:
# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80286628e4aae8e157.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
